# Pump it Up: Data Mining the Water Table
Predicting whether water pumps in Tanzania are faulty given a feature dataset

## Authors: Annie Zheng, Gligorco Vasilev, Diego Fernandez

### Sections:
1. Business Understanding
2. Data Understanding
3. Data Preparation
4. Modeling
5. Evaluation